In [1]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.linalg import lstsq
from scipy.optimize import leastsq

from functions_new_parameters import cov_func
from functions_new_parameters import generate_parallax_signal_np
from functions_new_parameters import signal_func_np
from functions_new_parameters import normalized_residuals_np

# %matplotlib inline
# %config InlineBackend.figure_format = "retina"

# # %run -i 'fitting_scriptfile.py'
# %run -i 'np_fitting_scipy.py'

In [2]:
times = np.linspace(0, 4.2, 4)

alpha0, delta0 = 1, 0.3

truepars = np.array((0,                       # alpha                                     [rad]
                     0,                       # delta                                     [rad]
                     2.3084641853871365e-07,  # proper motion in RA direction  (mu alpha) [rad/year]
                     1.770935480191023e-07,   # proper motion in Dec direction (mu delta) [rad/year]
                     9.699321049402031e-08))  # parallax                                  [rad]

sigma_err = (1e-5*np.pi/180/60/60)*5

In [3]:
pm_ra_true, prlx_ra_true, true_ra, pm_dec_true, prlx_dec_true, true_dec = signal_func_np(truepars, alpha0, delta0, times)

ra_obs  = true_ra   +  np.random.normal(0, sigma_err, len(true_ra)) 
dec_obs = true_dec  +  np.random.normal(0, sigma_err, len(true_dec)) 

yobs = np.concatenate((ra_obs , dec_obs))

In [4]:
guess = truepars * (1 + np.random.uniform(0,0.0001))
best, cov, _ , _ , _ = leastsq(normalized_residuals_np, guess, args=(alpha0, delta0, sigma_err, ra_obs, dec_obs, times), full_output=1)

In [5]:
# correction_term = []

# for i in range(2*len(times)):
#     if i < len(times):
#         correction_term = np.append(correction_term, alpha0) 
#     if i >= len(times):
#         correction_term = np.append(correction_term, delta0)
        
# print(correction_term)

In [6]:
times_double = np.concatenate((times, times))

params = ['Delta alpha 0', 'Delta delta 0', 'mu alpha', 'mu delta', 'parallax']

parallax_ra, parallax_dec = generate_parallax_signal_np(alpha0, delta0, 1, times)
parallax_both = np.concatenate((parallax_ra, parallax_dec))

M = np.zeros((2*len(times), len(params)))

for i in range(2*len(times)):
    if i < len(times):
        M[i,0] = 1
        M[i,1] = 0
        M[i,2] = times[i]
        M[i,3] = 0
        M[i,4] = parallax_ra[i]
    if i >= len(times):
        M[i,0] = 0
        M[i,1] = 1
        M[i,2] = 0
        M[i,3] = times_double[i]
        M[i,4] = parallax_both[i]
print(M)

cov_part2 = cov_func(M, sigma_err, print_cov=False)

x,  res,  rank,  s  = lstsq(M, yobs)

[[ 1.          0.          0.          0.          0.8808111 ]
 [ 1.          0.          1.4         0.         -1.04502033]
 [ 1.          0.          2.8         0.          0.81006731]
 [ 1.          0.          4.2         0.         -0.26569612]
 [ 0.          1.          0.          0.          0.15967025]
 [ 0.          1.          0.          1.4         0.0169896 ]
 [ 0.          1.          0.          2.8        -0.18716   ]
 [ 0.          1.          0.          4.2         0.28584164]]


In [7]:
_,_,ra1,_,_,dec1 =  signal_func_np(truepars,alpha0,delta0,times)

In [8]:
ra1[0]

8.543269652432592e-08

In [9]:
#parameters = ['Delta alpha0', 'Delta delta0', 'mu_alpha', 'mu_delta', 'parallax']

resids_test = lambda pars: (M @ np.array(pars)-(yobs))/sigma_err


# resids_alpha = lambda pars:  M_alpha @ np.array([pars[0],pars[2],pars[4]])-(ra_obs-alpha0)
# resids_delta = lambda pars:  M_alpha @ np.array([pars[1],pars[3],pars[4]])-(dec_obs-delta0)
# resids = lambda pars: np.concatenate((resids_alpha(pars),resids_delta(pars)))

x_test,cov_x,_,_,_ = leastsq(resids_test, guess, full_output=1)

# plt.plot(times, M_delta @  np.array([x[1],x[3],x[4]]) )
# plt.plot(times,(dec_obs-delta0))
# plt.show()

In [10]:
print("Part 1: Using scipy.optimize.leastsq")
for i in range(0, len(best)):
    print(params[i],(13-len(params[i]))*' ','=', best[i])

print(" ")
print("Part 2: Using scipy.linalg.lstsq")
for i in range(len(x)):
    print(params[i],  (13-len(params[i])) *' ','=', x[i])
    
print(" ")
print("Test:")
for i in range(len(x_test)):
    print(params[i],  (13-len(params[i])) *' ','=', x_test[i])

Part 1: Using scipy.optimize.leastsq
Delta alpha 0  = -7.619357917971556e-11
Delta delta 0  = 9.42064548493814e-11
mu alpha       = 2.308490788489521e-07
mu delta       = 1.770305162664979e-07
parallax       = 9.690837708717299e-08
 
Part 2: Using scipy.linalg.lstsq
Delta alpha 0  = -7.619358226064033e-11
Delta delta 0  = 9.420645334696915e-11
mu alpha       = 2.3084907885012708e-07
mu delta       = 1.770305162661772e-07
parallax       = 9.690837709130721e-08
 
Test:
Delta alpha 0  = -7.619357804378614e-11
Delta delta 0  = 9.420645490641554e-11
mu alpha       = 2.3084907884867743e-07
mu delta       = 1.7703051626651456e-07
parallax       = 9.690837708583669e-08


In [11]:
print("Part 1:")
print(cov)
print(" ")
print("Part 2:")
print(cov_part2)
print("Test")
print(cov_x)

Part 1:
[[ 4.36751512e-20  3.26571037e-22 -1.34561352e-20  9.53045816e-23
  -7.65197473e-21]
 [ 3.26571037e-22  4.11747495e-20 -1.11082970e-22 -1.25799774e-20
  -9.81505747e-22]
 [-1.34561352e-20 -1.11082970e-22  6.28999657e-21 -3.24178043e-23
   2.60281526e-21]
 [ 9.53045816e-23 -1.25799774e-20 -3.24178043e-23  5.99991795e-21
  -2.86436897e-22]
 [-7.65197473e-21 -9.81505747e-22  2.60281526e-21 -2.86436897e-22
   2.29979280e-20]]
 
Part 2:
[[ 4.36784321e-20  3.26568933e-22 -1.34576028e-20  9.52957857e-23
  -7.65144522e-21]
 [ 3.26568933e-22  4.11746469e-20 -1.11086398e-22 -1.25794343e-20
  -9.81555286e-22]
 [-1.34576028e-20 -1.11086398e-22  6.29058945e-21 -3.24160216e-23
   2.60273223e-21]
 [ 9.52957857e-23 -1.25794343e-20 -3.24160216e-23  5.99959553e-21
  -2.86426762e-22]
 [-7.65144522e-21 -9.81555286e-22  2.60273223e-21 -2.86426762e-22
   2.29976453e-20]]
Test
[[ 4.36798979e-20  3.26561696e-22 -1.34583582e-20  9.52916429e-23
  -7.65111042e-21]
 [ 3.26561696e-22  4.11741352e-20 -1.110